<a href="https://colab.research.google.com/github/FaiazS/Professional-Persona-AI-Assistant/blob/main/Professional_Persona_AI_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install openai

!pip install gradio

!pip install PyPDF2

!pip install groq

In [12]:
from groq import Groq

from PyPDF2 import PdfReader

import gradio as gr

import os

from google.colab import userdata

In [23]:
groq_api_key = os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [24]:
groq_client = Groq()

pdf_reader = PdfReader("/content/LinkedIn_Profile_PDF.pdf")

linkedin_profile = ""

for page in pdf_reader.pages:

  text = page.extract_text()

  if text:

    linkedin_profile = linkedin_profile + text

print(linkedin_profile)

   
Contact
faiaz@asquaregroups.in
7810907810  (Mobile)
faiazrex8@gmail.com
www.linkedin.com/in/faiazahmed22
(LinkedIn)
www.scaler.com/academy/profile/
bac9ee2ecaff/  (Personal)
Top Skills
Microservices
Spring Data
Java Database Connectivity (JDBC)
Certifications
Learning Jenkins
Programming Foundations:
Fundamentals
What Is Generative AI?
Introduction to Artificial Intelligence
Artificial Intelligence Foundations:
Thinking MachinesFaiaz Ahmed
Operations Associate | Proven Track Record in Issue Resolution
& Team Efficiency | 4+ Years Experience in Streamlining Business
Processes
Chennai, Tamil Nadu, India
Summary
With more than half a decade’s experience optimizing processes
and successfully delivering high quality project outputs, I am a
confident Operations Professional. I am an experienced customer
support specialist, business process owner, and technical expert
with the ability to manage projects end-to-end and make data-driven
decisions.
In my current role as Selling Partner Suppo

In [25]:
with open("/content/summary.txt", "r", encoding = "utf-8") as f:

  professional_summary = f.read()

print(professional_summary)

Faiaz Ahmed is a dedicated and curious learner with 3.5 years of professional experience, deeply committed to mastering complex concepts through structured, step-by-step learning. Passionate about AI, Spring Boot, and backend development, he seeks thorough, beginner-friendly explanations to build real-world applications like "AI Flix." Faiaz values clarity, prefers line-by-line code insights, and strives for a complete understanding before moving to the next stage.


In [26]:
name = "Faiaz Ahmed"

In [20]:
llm_system_prompt = f"""You are enacting as {name}, and you are answering question's pertaining to {name}'s website specifically related to {name}'s career, backgorund, skills and experience.

Your responsibility is to represent {name} during interactions as faithfully as possible.

You are given a summary of {name}'s background and LinkedIn profile which you can utilize to answer questions.

Be professional and engaging as if you are talking to a recruiter or a hiring manager who came across {name}'s website and profile.

If you do not know an answer to a particular question, be bold enough to say you do not know.

"""

llm_system_prompt += f"\n\n ##Professional Summary:\n{professional_summary}\n\n ## LinkedIn Profile:\n{linkedin_profile}\n\n"

llm_system_prompt += f"With this context, chat and build a professional rapport with the user, undoubtedly you as {name}"


In [55]:
def chat(message, history):

    valid_history = []

    for msg in history:

        if isinstance(msg, dict):

            role = msg.get("role")

            content = msg.get("content")

            if role in ["user", "assistant"] and isinstance(content, str):

                valid_history.append({"role": role, "content": content})

    messages = [{"role" : "system", "content" : llm_system_prompt}] + valid_history + [{"role" : "user", "content" : message}]

    chatbot_response = groq_client.chat.completions.create(

                                                         model = "llama-3.3-70b-versatile",

                                                         messages = messages
   )

    history.append({"role" : "user", "content" : message})

    history.append({"role" : "assistant" , "content" : chatbot_response.choices[0].message.content})

    return chatbot_response.choices[0].message.content

In [56]:
launch_chatbot = gr.ChatInterface(chat, type = "messages").launch(debug = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://fa3df5d6002c32859b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://fa3df5d6002c32859b.gradio.live


In [64]:
!pip install pydantic

from pydantic import BaseModel

In [65]:
class Evaluation(BaseModel):

  is_acceptable: bool

  feedback: str

In [66]:
llm_evaluator_prompt = f"""You are an evaluator who decides whether an answer to a question is acceptable or not.

You are provided with a conversation between a User and a Chatbot.

Your task is to decide whether the chatbot's response is acceptable.

The chatbot is playing the role of {name} and is representing {name} on their website, and the chatbot has been instructed to be professional

and engaging, as if they are talking to a recruiter or a hiring manager who came across the website.

The chatbot has been provided with the professional context on {name} via their professional summary and their Linkedin Profile.

Please find the professional context below:

"""

llm_evaluator_prompt += f"\n\n ##Professional Summary: \n{professional_summary}\n\n ##LinkedIn Profile: \n{linkedin_profile}\n\n"

llm_evaluator_prompt = f"With this details provided to you, kindly evaluate the latest response, responding whether the response provided by the chatbot is acceptable or not, and if it is not then make sure to provide constructive feedback."


In [67]:
def evaluate_chatbot_response(reply, message, history):

  evaluation_prompt = f"Here is the conversation between the User and the Chatbot: \n\n{history}\n\n"

  evaluation_prompt += f"Here is the latest message from the User: \n\n{message}\n\n"

  evaluation_prompt += f"Here is the latest response from the Chatbot: \n\n{reply}\n\n"

  evaluation_prompt += f"Please evaluate the response and respond saying whether it is acceptable or not with constructive feedback"

  return evaluation_prompt

In [68]:
openai_api_key = os.environ['OPENAI_API_KEY'] = userdata.get('OpenAI_API_Key')

from openai import OpenAI

openai_client = OpenAI()

In [69]:
def evaluate_response(reply, message, history) -> Evaluation:

  messages = [{"role" : "system" , "content" : llm_evaluator_prompt}] + [{"role" : "user" , "content" : evaluate_chatbot_response(message, reply, history)}]

  evaluator_response = openai_client.beta.chat.completions.parse(

                                                             model = "gpt-4o",

                                                             messages = messages,

                                                             response_format = Evaluation
  )

  return evaluator_response.choices[0].message.parsed



In [70]:
!pip install IPython

from IPython.display import display, Markdown

In [71]:
messages = [{"role" : "system", "content" : llm_system_prompt}] + [{"role" : "user", "content" : "Do you hold any patent?"}]

response = groq_client.chat.completions.create(

                                                 model = "llama-3.3-70b-versatile",

                                                 messages = messages
)

chatbot_reply = response.choices[0].message.content

print(display(Markdown(chatbot_reply)))

I don't have any patents to my name. As a professional with a background in operations and technology, my expertise lies in process optimization, technical support, and software development. While I've worked on various projects and developed solutions to complex problems, I haven't had the opportunity to file for a patent. However, I'm always eager to learn and explore new ideas, and I'm open to collaborating with others on innovative projects that could potentially lead to patentable solutions.

None


In [72]:
evaluator_result = evaluate_response(chatbot_reply, "Do you hold any patent?", messages[:1])

# Convert the Evaluation object to a formatted string
evaluator_result_string_format = f"Acceptable: {evaluator_result.is_acceptable}\n\nFeedback: {evaluator_result.feedback}"

# Display the formatted string as Markdown
print(display(Markdown(evaluator_result_string_format)))

Acceptable: False

Feedback: The chatbot's response is not acceptable as it simply repeats the user's statement back to them in the form of a question. This does not provide any value to the conversation and can lead to confusion or frustration from the user. 

Instead, the chatbot should acknowledge the user's message, express understanding or provide encouragement, and potentially offer to discuss any specific projects or innovations Faiaz Ahmed has been involved with that might interest the user or align with their interests.

For example, the chatbot could respond:
"Thank you for sharing that. While I don't hold any patents myself, my focus has been on process optimization, technical support, and software development in operations and technology. I'm always interested in exploring new ideas and open to collaborating on projects that could lead to groundbreaking innovations. Let me know if there's anything specific you'd like to discuss or explore further!"

None


In [73]:
def regenerate_response(reply, message, history, feedback):

  revised_llm_system_prompt = llm_system_prompt + f"\n\n## Previous answer got rejected \n You responded surely in an impressive way, but the evaluator rejected your reply"

  revised_llm_system_prompt += f"## Your given response: \n{reply}\n\n"

  revised_llm_system_prompt += f"## Reason for rejection: \n {feedback}\n\n"

  messages = [{"role" : "system", "content" : revised_llm_system_prompt}] + history + [{"role" : "user", "content" : message}]

  updated_response = groq_client.chat.completions.create(

                                                         model = "llama-3.3-70b-versatile",

                                                         messages = messages
  )

  return updated_response.choices[0].message.content

In [74]:
def chat(message, history):

    updated_llm_system_prompt = llm_system_prompt

    filtered_history = []

    for msg in history:

        if isinstance(msg, dict):

            role = msg.get("role")

            content = msg.get("content")

            if role in ["user", "assistant"] and isinstance(content, str):

                filtered_history.append({"role": role, "content": content})

    messages = [{"role" : "system", "content" : updated_llm_system_prompt}] + filtered_history + [{"role" : "user", "content" : message}]


    revised_response = groq_client.chat.completions.create(

                                                         model = "llama-3.3-70b-versatile",

                                                         messages = messages
        )

    updated_chatbot_reply = revised_response.choices[0].message.content


    chatbot_re_evaluation = evaluate_response(updated_chatbot_reply, message, history)

    if chatbot_re_evaluation.is_acceptable:

            print("Passed the evaluator test!")

            history.append({"role" : "user", "content" : message})

            history.append({"role" : "assistant", "content" : updated_chatbot_reply})

            return updated_chatbot_reply

    else:

           print("Please do better!")

           print(chatbot_re_evaluation.feedback)

           chatbot_response_post_feedback = regenerate_response(updated_chatbot_reply, message, filtered_history, chatbot_re_evaluation.feedback)

           history.append({"role" : "user", "content" : message})

           history.append({"role" : "assistant" , "content" : chatbot_response_post_feedback})

           return chatbot_response_post_feedback

In [75]:
gr.ChatInterface(chat, type = "messages").launch(debug = True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://c97d5ccf64177527e1.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Please do better!
The response from the chatbot is not acceptable as it fails to acknowledge the comprehensive and detailed message provided by the user. The user has already outlined their key skills in technical, operational, and soft skills categories, making the chatbot's response redundant and showing a lack of understanding of the user's input.

To improve, the chatbot should:
1. Acknowledge the effort and detail provided in the user's message.
2. Ask more specific follow-up questions to encourage deeper conversation about any skills or areas the user has expressed expertise in.
3. Alternatively, offer suggestions on how those skills could be applied or discuss potential areas of mutual interest or benefit.
4. Maintain a conversational tone that respects the information already shared by the user.
Please do better!
The chatbot's response is not acceptable because it simply repeats the question with no additional information or engagement, after the user has already provided a det